In [1]:
import numpy as np
import pandas as pd

threshold = 0.5

In [2]:
train_embedding = np.load('./data/BERT_encoding/train_encode_tuned.npy')

In [3]:
dev_embedding = np.load('./data/BERT_encoding/dev_encode_tuned.npy')
dev_embedding = dev_embedding[0: 1000]

In [4]:
train_embedding.shape

(291862, 768)

In [5]:
dev_embedding.shape

(1000, 768)

In [6]:
dev_data = pd.read_csv('./data/dev_set_v1.txt', sep='\t', header=None)
dev_data = dev_data[0: 1000]

In [7]:
train_data = pd.read_csv('./data/train_set_v1.txt', sep='\t', header=None)

### Load keras model

In [8]:

import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import Adam


model = Sequential()
model.add(Dense(units=768, activation='relu', input_dim=1536))
# model.add(Dense(units=50, activation='relu', input_dim=x_train.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(units=2, activation='softmax'))
# optimizer = Adam(lr=0.01)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam', metrics=['accuracy'])

model.summary()

model.load_weights("./Train_pair/best_pair_new.hdf5")

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 768)               1180416   
_________________________________________________________________
dropout_1 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1538      
Total params: 1,181,954
Trainable params: 1,181,954
Non-trainable params: 0
_________________________________________________________________


In [9]:
%%time
top_k = 10

# pairs = np.concatenate((np.repeat([dev_embedding[0]], [len(train_embedding)], axis=0), train_embedding), axis=1)
# result = model.predict(pairs, batch_size=1024, verbose=1)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 6.44 µs


In [ ]:
top_k_idx=result[:,1].argsort()[::-1][0:top_k]

In [ ]:
np.array(train_data[0])[top_k_idx]

In [ ]:
counts = np.bincount(np.array(train_data[0])[top_k_idx])
np.argmax(counts)

In [11]:
correct = 0

for i in range(len(dev_data)):
    pairs = np.concatenate((np.repeat([dev_embedding[i]], [len(train_embedding)], axis=0), train_embedding), axis=1)
    result = model.predict(pairs, batch_size=1024, verbose=1)
#     max_index = np.argmax(result[:,1])
    top_k_idx=result[:,1].argsort()[::-1][0:top_k]
    np.array(train_data[0])[top_k_idx]
    counts = np.bincount(np.array(train_data[0])[top_k_idx])
    
    if dev_data[0][i] == np.argmax(counts):
        print("correct")
        correct += 1

correct

291862/291862 [==============================] - 1s 4us/step


KeyboardInterrupt: 